In [77]:
import pandas as pd
import pandasql as ps

import os

cwd = os.getcwd()
filepath = cwd + '/data/combined_ts.csv'
populationfilepath = cwd + '/data/populationdata.csv'

df = pd.read_csv(filepath, delimiter=',')
population_df = pd.read_csv(populationfilepath, delimiter='|')

In [78]:
df = df.fillna(0)
print(df.head(5))

   Unnamed: 0 abbr  seconds_since_Epoch  tested  positive  deaths
0           0   AK           1583651326    20.0       0.0     0.0
1           1   AK           1583700172    23.0       0.0     0.0
2           2   AK           1583790457    32.0       0.0     0.0
3           3   AK           1583880060    47.0       0.0     0.0
4           4   AK           1583981097    60.0       0.0     0.0


In [79]:
#pm means per million
query1 = """
SELECT q1.*
    , q2.population 
    , 1000000/q2.population as multipliler
    , tested * 1000000/q2.population as tested_pm
    , positive * 1000000/q2.population as positive_pm
    , deaths * 1000000/q2.population as deaths_pm
FROM df as q1
left outer join population_df as q2
on q1.abbr = q2.abbr
"""
 


In [80]:
result = ps.sqldf(query1, locals())

result.head(15)

,Unnamed: 0,abbr,seconds_since_Epoch,tested,positive,deaths,population,multipliler,tested_pm,positive_pm,deaths_pm
0,0,AK,1583651326,20.0,0.0,0.0,4903185,0,4.078981,0.000000,0.0
1,1,AK,1583700172,23.0,0.0,0.0,4903185,0,4.690829,0.000000,0.0
2,2,AK,1583790457,32.0,0.0,0.0,4903185,0,6.526370,0.000000,0.0
3,3,AK,1583880060,47.0,0.0,0.0,4903185,0,9.585606,0.000000,0.0
4,4,AK,1583981097,60.0,0.0,0.0,4903185,0,12.236944,0.000000,0.0
5,5,AK,1584110139,60.0,1.0,0.0,4903185,0,12.236944,0.203949,0.0
6,6,AK,1584137412,144.0,1.0,0.0,4903185,0,29.368665,0.203949,0.0
7,7,AK,1584396920,251.0,1.0,0.0,4903185,0,51.191216,0.203949,0.0
8,8,AK,1584422731,337.0,3.0,0.0,4903185,0,68.730835,0.611847,0.0
9,9,AK,1584515664,438.0,6.0,0.0,4903185,0,89.329691,1.223694,0.0


In [81]:
# Now we need to calculate the daily differences
query2 = """
SELECT q1.*
    , tested - coalesce(lag(tested,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_tests
    , positive - coalesce(lag(positive,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_positive
    , deaths - coalesce(lag(deaths,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_deaths
    , tested_pm - coalesce(lag(tested_pm,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_tests_pm
    , positive_pm - coalesce(lag(positive_pm,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_positive_pm
    , deaths_pm - coalesce(lag(deaths_pm,1) over (partition by abbr order by seconds_since_Epoch ),0) as daily_deaths_pm
FROM result as q1
"""

In [82]:
dailydiff = ps.sqldf(query2, locals())

dailydiff.head(15)

,Unnamed: 0,abbr,seconds_since_Epoch,tested,positive,deaths,population,multipliler,tested_pm,positive_pm,deaths_pm,daily_tests,daily_positive,daily_deaths,daily_tests_pm,daily_positive_pm,daily_deaths_pm
0,0,AK,1583651326,20.0,0.0,0.0,4903185,0,4.078981,0.000000,0.0,20.0,0.0,0.0,4.078981,0.000000,0.0
1,1,AK,1583700172,23.0,0.0,0.0,4903185,0,4.690829,0.000000,0.0,3.0,0.0,0.0,0.611847,0.000000,0.0
2,2,AK,1583790457,32.0,0.0,0.0,4903185,0,6.526370,0.000000,0.0,9.0,0.0,0.0,1.835542,0.000000,0.0
3,3,AK,1583880060,47.0,0.0,0.0,4903185,0,9.585606,0.000000,0.0,15.0,0.0,0.0,3.059236,0.000000,0.0
4,4,AK,1583981097,60.0,0.0,0.0,4903185,0,12.236944,0.000000,0.0,13.0,0.0,0.0,2.651338,0.000000,0.0
5,5,AK,1584110139,60.0,1.0,0.0,4903185,0,12.236944,0.203949,0.0,0.0,1.0,0.0,0.000000,0.203949,0.0
6,6,AK,1584137412,144.0,1.0,0.0,4903185,0,29.368665,0.203949,0.0,84.0,0.0,0.0,17.131722,0.000000,0.0
7,7,AK,1584396920,251.0,1.0,0.0,4903185,0,51.191216,0.203949,0.0,107.0,0.0,0.0,21.822550,0.000000,0.0
8,8,AK,1584422731,337.0,3.0,0.0,4903185,0,68.730835,0.611847,0.0,86.0,2.0,0.0,17.539620,0.407898,0.0
9,9,AK,1584515664,438.0,6.0,0.0,4903185,0,89.329691,1.223694,0.0,101.0,3.0,0.0,20.598856,0.611847,0.0


In [83]:
dailydiff['daily_tests_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_tests'].mean().reset_index(drop=True)
dailydiff['daily_positive_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_positive'].mean().reset_index(drop=True)
dailydiff['daily_deaths_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_deaths'].mean().reset_index(drop=True)
dailydiff['daily_tests_pm_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_tests_pm'].mean().reset_index(drop=True)
dailydiff['daily_positive_pm_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_positive_pm'].mean().reset_index(drop=True)
dailydiff['daily_deaths_pm_7day_mean']=dailydiff.groupby('abbr').rolling(7)['daily_deaths_pm'].mean().reset_index(drop=True)

runningavg = dailydiff.fillna(0)

runningavg.head(115)

,Unnamed: 0,abbr,seconds_since_Epoch,tested,positive,deaths,population,multipliler,tested_pm,positive_pm,...,daily_deaths,daily_tests_pm,daily_positive_pm,daily_deaths_pm,daily_tests_7day_mean,daily_positive_7day_mean,daily_deaths_7day_mean,daily_tests_pm_7day_mean,daily_positive_pm_7day_mean,daily_deaths_pm_7day_mean
0,0,AK,1583651326,20.0,0.0,0.0,4903185,0,4.078981,0.000000,...,0.0,4.078981,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,1,AK,1583700172,23.0,0.0,0.0,4903185,0,4.690829,0.000000,...,0.0,0.611847,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,2,AK,1583790457,32.0,0.0,0.0,4903185,0,6.526370,0.000000,...,0.0,1.835542,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,3,AK,1583880060,47.0,0.0,0.0,4903185,0,9.585606,0.000000,...,0.0,3.059236,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,4,AK,1583981097,60.0,0.0,0.0,4903185,0,12.236944,0.000000,...,0.0,2.651338,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,110,AK,1590270482,42351.0,408.0,10.0,4903185,0,8637.446884,83.211219,...,0.0,184.573905,0.815796,0.0,1100.0,1.714286,0.0,224.343972,0.349627,0.0
111,0,AL,1583651326,0.0,0.0,0.0,731545,1,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
112,1,AL,1583900198,10.0,0.0,0.0,731545,1,13.669699,0.000000,...,0.0,13.669699,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
113,2,AL,1584110139,12.0,1.0,0.0,731545,1,16.403639,1.366970,...,0.0,2.733940,1.366970,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [84]:
query3 = """
SELECT q1.*
    , rank() over (partition by seconds_since_Epoch order by daily_tests_7day_mean desc) as daily_test_rk
    , rank() over (partition by seconds_since_Epoch order by daily_positive_7day_mean desc) as daily_positive_rk
    , rank() over (partition by seconds_since_Epoch order by daily_deaths_7day_mean desc) as daily_deaths_rk
    , rank() over (partition by seconds_since_Epoch order by daily_tests_pm_7day_mean desc) as daily_test_pm_rk
    , rank() over (partition by seconds_since_Epoch order by daily_positive_pm_7day_mean desc) as daily_positive_pm_rk
    , rank() over (partition by seconds_since_Epoch order by daily_deaths_pm_7day_mean desc) as daily_deaths_pm_rk
FROM runningavg as q1
"""

In [86]:
final = ps.sqldf(query3, locals())

final.head(-51)

,Unnamed: 0,abbr,seconds_since_Epoch,tested,positive,deaths,population,multipliler,tested_pm,positive_pm,...,daily_deaths_7day_mean,daily_tests_pm_7day_mean,daily_positive_pm_7day_mean,daily_deaths_pm_7day_mean,daily_test_rk,daily_positive_rk,daily_deaths_rk,daily_test_pm_rk,daily_positive_pm_rk,daily_deaths_pm_rk
0,0,AK,1583651326,20.0,0.0,0.0,4903185,0,4.078981,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,1,1,1,1,1,1
1,0,AL,1583651326,0.0,0.0,0.0,731545,1,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,1,1,1,1,1,1
2,0,AR,1583651326,6.0,0.0,0.0,7278717,0,0.824321,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,1,1,1,1,1,1
3,0,AZ,1583651326,56.0,5.0,0.0,3017804,0,18.556540,1.656834,...,0.000000,0.000000,0.000000,0.000000e+00,1,1,1,1,1,1
4,0,CA,1583651326,515.0,88.0,1.0,39512223,0,13.033941,2.227159,...,0.000000,0.000000,0.000000,0.000000e+00,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,89,ND,1590220143,62830.0,2317.0,52.0,3080156,0,20398.317488,752.234627,...,1.428571,515.511923,25.787191,4.637984e-01,17,16,18,15,12,17
5083,73,SD,1590220143,32344.0,4356.0,50.0,884659,1,36560.979993,4923.931142,...,1.571429,1049.637689,100.765218,1.776310e+00,18,15,17,10,8,9
5084,102,MT,1590220143,31857.0,479.0,16.0,1068778,0,29806.938391,448.175393,...,0.000000,860.662498,1.737632,1.268826e-16,19,23,23,12,22,23
5085,101,ID,1590220143,40370.0,2595.0,79.0,12671821,0,3185.808890,204.785090,...,0.857143,44.124902,1.984155,6.764165e-02,20,20,19,23,21,21
